In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [25]:
ratings = pd.read_csv('./movie_data/ratings.csv')

In [26]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [27]:
!dir movie_data

 E 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 4D42-3DB2

 E:\NewDataScience\04_AI\04_응용\02_추천시스템\01_협업필터링\movie_data 디렉터리

2023-01-30  오후 05:06    <DIR>          .
2023-01-30  오후 05:06    <DIR>          ..
2019-09-21  오전 10:30         2,438,266 ratings_small.csv
2023-01-30  오후 05:16           674,990 ratings.csv.zip
2023-01-30  오후 05:17           458,390 movies.csv
2019-09-21  오후 03:25         2,438,266 ratings.csv
               4개 파일           6,009,912 바이트
               2개 디렉터리  95,158,403,072 바이트 남음


In [28]:
movies = pd.read_csv('./movie_data/movies.csv')

In [29]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [30]:
ratings_movies = pd.merge(ratings, movies, on = 'movieId')

In [31]:
ratings_movies.head()

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,7,31,3.0,851868750,Dangerous Minds (1995),Drama
2,31,31,4.0,1273541953,Dangerous Minds (1995),Drama
3,32,31,4.0,834828440,Dangerous Minds (1995),Drama
4,36,31,3.0,847057202,Dangerous Minds (1995),Drama


In [32]:
ratings_matrix = ratings.pivot_table(index=['movieId'],columns=['userId'],values='rating').reset_index(drop=True)
ratings_matrix.fillna( 0, inplace = True )
ratings_matrix.head(15)

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,4.0,0.0,...,0.0,4.0,3.5,0.0,0.0,0.0,0.0,0.0,4.0,5.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,5.0,4.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,4.0,0.0,4.0,0.0,0.0,3.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0


In [33]:
movie_similarity=cosine_similarity(ratings_matrix)
np.fill_diagonal( movie_similarity, 0 ) 
movie_similarity

array([[0.        , 0.39451145, 0.30651588, ..., 0.        , 0.        ,
        0.05582876],
       [0.39451145, 0.        , 0.21749153, ..., 0.        , 0.        ,
        0.        ],
       [0.30651588, 0.21749153, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.05582876, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [34]:
ratings_matrix = pd.DataFrame( movie_similarity )
ratings_matrix.head(15)

,0,1,2,3,4,5,6,7,8,9,...,9056,9057,9058,9059,9060,9061,9062,9063,9064,9065
0,0.000000,0.394511,0.306516,0.133614,0.245102,0.377086,0.278629,0.063031,0.117499,0.310689,...,0.055829,0.031902,0.079755,0.079755,0.079755,0.079755,0.079755,0.0,0.0,0.055829
1,0.394511,0.000000,0.217492,0.164651,0.278476,0.222003,0.207299,0.223524,0.113669,0.418124,...,0.000000,0.055038,0.068797,0.082557,0.082557,0.137594,0.068797,0.0,0.0,0.000000
2,0.306516,0.217492,0.000000,0.177012,0.370732,0.247499,0.435648,0.127574,0.306717,0.191255,...,0.000000,0.000000,0.000000,0.116226,0.116226,0.000000,0.000000,0.0,0.0,0.000000
3,0.133614,0.164651,0.177012,0.000000,0.179556,0.072518,0.184626,0.501513,0.254630,0.111447,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
4,0.245102,0.278476,0.370732,0.179556,0.000000,0.272645,0.388476,0.194113,0.367941,0.246846,...,0.000000,0.176845,0.000000,0.117897,0.117897,0.000000,0.000000,0.0,0.0,0.000000
5,0.377086,0.222003,0.247499,0.072518,0.272645,0.000000,0.278855,0.097561,0.248155,0.307948,...,0.061724,0.098758,0.111103,0.000000,0.000000,0.000000,0.111103,0.0,0.0,0.061724
6,0.278629,0.207299,0.435648,0.184626,0.388476,0.278855,0.000000,0.196091,0.349827,0.177425,...,0.079399,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.079399
7,0.063031,0.223524,0.127574,0.501513,0.194113,0.097561,0.196091,0.000000,0.264477,0.042169,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
8,0.117499,0.113669,0.306717,0.254630,0.367941,0.248155,0.349827,0.264477,0.000000,0.130475,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
9,0.310689,0.418124,0.191255,0.111447,0.246846,0.307948,0.177425,0.042169,0.130475,0.000000,...,0.000000,0.076835,0.076835,0.102446,0.102446,0.000000,0.076835,0.0,0.0,0.000000


In [ ]:
try:
    #user_inp=input('Enter the reference movie title based on which recommendations are to be made: ')
    user_inp="Jumanji (1995)"
    inp=movies[movies['title']==user_inp].index.tolist()
    inp=inp[0]
    
    movies['similarity'] = ratings_matrix.iloc[inp]
    movies.head(5)
    
except:
    print("Sorry, the movie is not in the database!")
    

In [36]:
print("Recommended movies based on your choice of ",user_inp ,": \n", 
      movies.sort_values( ["similarity"], ascending = False )[1:10])

Recommended movies based on your choice of  Jumanji (1995) : 
      movieId                              title  \
328      364              Lion King, The (1994)   
283      317           Santa Clause, The (1994)   
331      367                   Mask, The (1994)   
527      595        Beauty and the Beast (1991)   
521      588                     Aladdin (1992)   
309      344  Ace Ventura: Pet Detective (1994)   
520      587                       Ghost (1990)   
427      480               Jurassic Park (1993)   
341      377                       Speed (1994)   

                                              genres  similarity  
328  Adventure|Animation|Children|Drama|Musical|IMAX    0.530357  
283                             Comedy|Drama|Fantasy    0.505831  
331                      Action|Comedy|Crime|Fantasy    0.494605  
527  Animation|Children|Fantasy|Musical|Romance|IMAX    0.494124  
521      Adventure|Animation|Children|Comedy|Musical    0.493995  
309                     